# Sales Forecast Model

This notebook calculates forecast unit sales based on demand forecasts and available inventory (on-hand and on-order).

The forecast unit sales represents how much of the unit demand can be captured/covered by available inventory.

**Output columns:**
- SKU: Product SKU
- MONTH: Forecast month
- UNIT DEMAND: Forecasted unit demand
- FORECAST UNIT SALES: Unit demand that can be captured by available inventory
- MISSED DEMAND: Unit demand that cannot be captured due to insufficient inventory

## Setup and Imports

In [153]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

## Configuration

Set file paths here. All data files should be in the `data` folder.

In [154]:
# File paths (relative to project root)
data_folder = '../data'
demand_forecast_file = f'{data_folder}/CZ Demand Forecast Sample.csv'
on_hand_inventory_file = f'{data_folder}/on hand inventory_sample.csv'
on_order_file = f'{data_folder}/CZ On Order Sample Data.csv'
catalog_file = f'{data_folder}/cz_catalog_data.csv'

## Load Data Files

In [155]:
print("Loading data files...")

# Load demand forecast
demand_forecast = pd.read_csv(demand_forecast_file)
print(f"Demand forecast shape: {demand_forecast.shape}")
print(f"Demand forecast columns: {demand_forecast.columns.tolist()}")
print(f"\nDemand forecast sample:")
print(demand_forecast.head())

# Load on-hand inventory
on_hand_inventory = pd.read_csv(on_hand_inventory_file)
print(f"\nOn-hand inventory shape: {on_hand_inventory.shape}")
print(f"On-hand inventory columns: {on_hand_inventory.columns.tolist()}")
print(f"\nOn-hand inventory sample:")
print(on_hand_inventory.head())

# Load on-order inventory
on_order = pd.read_csv(on_order_file)
print(f"\nOn-order inventory shape: {on_order.shape}")
print(f"On-order inventory columns: {on_order.columns.tolist()}")
print(f"\nOn-order inventory sample:")
print(on_order.head())

# Load catalog data with lead times
catalog = pd.read_csv(catalog_file)
print(f"\nCatalog shape: {catalog.shape}")
print(f"Catalog columns: {catalog.columns.tolist()}")
print(f"\nCatalog sample (showing lead time fields):")
print(catalog[['SKU', 'PRODUCTION_TIME', 'TRANSIT_TIME', 'TOTAL_LEAD_TIME']].head(10))

Loading data files...
Demand forecast shape: (54376, 4)
Demand forecast columns: ['SKU', 'MONTH', 'UNIT DEMAND', 'UNIT SALES']

Demand forecast sample:
          SKU      MONTH  UNIT DEMAND  UNIT SALES
0   P20001-05  11/1/2025          458         489
1   P20001-24  11/1/2025          380         477
2   P20001-25  11/1/2025          345         384
3   P20001-23  11/1/2025          281         325
4  D104007-01  11/1/2025          126         302

On-hand inventory shape: (6109, 7)
On-hand inventory columns: ['SKU', 'ON_HAND_QTY', 'QTY_COMMITTED', 'QTY_BACKORDERED', 'UNFULFILLED_QTY', 'AVAILABLE_ON_HAND_QTY', 'CURRENT_INVENTORY_POSITION']

On-hand inventory sample:
                   SKU  ON_HAND_QTY  QTY_COMMITTED  QTY_BACKORDERED  \
0           R120041-01            0              0                0   
1  0118316-010-U-W-009            0              0                0   
2            R94001-03            0              0                0   
3  0118316-010-U-W-008            0      

## Data Cleaning and Preparation

In [156]:
# Clean and prepare demand forecast data
# Convert MONTH to datetime
demand_forecast['MONTH'] = pd.to_datetime(demand_forecast['MONTH'], errors='coerce')
# Ensure SKU is string
demand_forecast['SKU'] = demand_forecast['SKU'].astype(str)

# Remove any rows with missing critical data
demand_forecast = demand_forecast.dropna(subset=['SKU', 'MONTH', 'UNIT DEMAND'])
print(f"Demand forecast after cleaning: {demand_forecast.shape}")

# Clean on-hand inventory data
on_hand_inventory['SKU'] = on_hand_inventory['SKU'].astype(str)
# Fill missing values with 0 for numeric columns
numeric_cols = ['ON_HAND_QTY', 'AVAILABLE_ON_HAND_QTY', 'CURRENT_INVENTORY_POSITION']
for col in numeric_cols:
    if col in on_hand_inventory.columns:
        on_hand_inventory[col] = pd.to_numeric(on_hand_inventory[col], errors='coerce').fillna(0)
print(f"On-hand inventory after cleaning: {on_hand_inventory.shape}")

# Clean on-order inventory data
on_order['SKU'] = on_order['SKU'].astype(str)
# Parse Estimate Ship Date - handle the column name with spaces
ship_date_col = 'Estimate Ship Date Date'  # Based on the CSV header
on_order[ship_date_col] = pd.to_datetime(on_order[ship_date_col], errors='coerce')
# Fill missing quantities with 0
qty_col = 'Expected Shipment Quantity'
on_order[qty_col] = pd.to_numeric(on_order[qty_col], errors='coerce').fillna(0)
# Remove rows with missing SKU or ship date
on_order = on_order.dropna(subset=['SKU', ship_date_col])
print(f"On-order inventory after cleaning: {on_order.shape}")

# Clean catalog data
catalog['SKU'] = catalog['SKU'].astype(str)
# Extract lead time fields and convert to numeric
catalog['PRODUCTION_TIME'] = pd.to_numeric(catalog['PRODUCTION_TIME'], errors='coerce').fillna(0)
catalog['TRANSIT_TIME'] = pd.to_numeric(catalog['TRANSIT_TIME'], errors='coerce').fillna(0)
catalog['TOTAL_LEAD_TIME'] = pd.to_numeric(catalog['TOTAL_LEAD_TIME'], errors='coerce').fillna(0)
# Create a simplified catalog with SKU, lead times, planning status, and MTO/DROPSHIP flags
catalog_leadtimes = catalog[['SKU', 'PRODUCTION_TIME', 'TRANSIT_TIME', 'TOTAL_LEAD_TIME', 'PLANNING_STATUS', 'IS_MTO', 'IS_DROPSHIP']].copy()
# Clean PLANNING_STATUS - handle missing values and convert to uppercase for consistent matching
catalog_leadtimes['PLANNING_STATUS'] = catalog_leadtimes['PLANNING_STATUS'].fillna('').astype(str).str.upper().str.strip()
# Convert IS_MTO and IS_DROPSHIP to boolean (handle TRUE/FALSE strings)
catalog_leadtimes['IS_MTO'] = catalog_leadtimes['IS_MTO'].astype(str).str.upper().str.strip().isin(['TRUE', '1', 'YES'])
catalog_leadtimes['IS_DROPSHIP'] = catalog_leadtimes['IS_DROPSHIP'].astype(str).str.upper().str.strip().isin(['TRUE', '1', 'YES'])
print(f"\nCatalog lead times shape: {catalog_leadtimes.shape}")
print(f"SKUs with production time > 0: {(catalog_leadtimes['PRODUCTION_TIME'] > 0).sum()}")
print(f"SKUs with ARCHIVED status: {(catalog_leadtimes['PLANNING_STATUS'] == 'ARCHIVED').sum()}")
print(f"SKUs with DISCONTINUED status: {(catalog_leadtimes['PLANNING_STATUS'] == 'DISCONTINUED').sum()}")
print(f"SKUs with IS_MTO = TRUE: {catalog_leadtimes['IS_MTO'].sum()}")
print(f"SKUs with IS_DROPSHIP = TRUE: {catalog_leadtimes['IS_DROPSHIP'].sum()}")
print(f"SKUs with IS_MTO OR IS_DROPSHIP = TRUE: {(catalog_leadtimes['IS_MTO'] | catalog_leadtimes['IS_DROPSHIP']).sum()}")

Demand forecast after cleaning: (54376, 4)
On-hand inventory after cleaning: (6109, 7)
On-order inventory after cleaning: (772, 5)

Catalog lead times shape: (6109, 7)
SKUs with production time > 0: 1828
SKUs with ARCHIVED status: 1720
SKUs with DISCONTINUED status: 1130
SKUs with IS_MTO = TRUE: 1288
SKUs with IS_DROPSHIP = TRUE: 1815
SKUs with IS_MTO OR IS_DROPSHIP = TRUE: 1816


## Process On-Order Inventory by Month

On-order inventory becomes sellable after the Estimate Ship Date. We need to aggregate on-order quantities by SKU and the month they become available.

In [157]:
# Create a month column for on-order data based on Estimate Ship Date
on_order['SHIP_MONTH'] = on_order[ship_date_col].dt.to_period('M').dt.to_timestamp()

# Aggregate on-order quantities by SKU and month
on_order_monthly = on_order.groupby(['SKU', 'SHIP_MONTH'])[qty_col].sum().reset_index()
on_order_monthly = on_order_monthly.rename(columns={qty_col: 'ON_ORDER_QTY', 'SHIP_MONTH': 'MONTH'})

print(f"On-order inventory aggregated by SKU and month: {on_order_monthly.shape}")
print(f"\nSample on-order monthly data:")
print(on_order_monthly.head(10))

On-order inventory aggregated by SKU and month: (695, 3)

Sample on-order monthly data:
          SKU      MONTH  ON_ORDER_QTY
0  B100001-01 2026-05-01            90
1  B100002-01 2026-05-01            70
2  B100002-02 2026-05-01            60
3  B100005-01 2026-05-01            10
4  B100005-02 2026-05-01            25
5  B116001-01 2025-12-01            57
6  B116001-02 2025-12-01            55
7  B120001-01 2026-01-01            35
8  B120001-02 2026-01-01            65
9  B120002-01 2026-01-01            35


## Calculate Available Inventory by Month

For each SKU and month, we need to calculate:
- Starting available on-hand inventory
- Cumulative on-order inventory that has arrived by that month
- Total available inventory = available on-hand + cumulative on-order

In [158]:
# Get unique SKUs and months from demand forecast
unique_skus = demand_forecast['SKU'].unique()
unique_months = sorted(demand_forecast['MONTH'].unique())

print(f"Unique SKUs: {len(unique_skus)}")
print(f"Unique months: {len(unique_months)}")
print(f"Month range: {unique_months[0]} to {unique_months[-1]}")

# Create a base dataframe with all SKU-month combinations from demand forecast
forecast_base = demand_forecast[['SKU', 'MONTH', 'UNIT DEMAND']].copy()

# Merge with on-hand inventory to get starting available inventory
# We'll use AVAILABLE_ON_HAND_QTY as the starting point
forecast_base = forecast_base.merge(
    on_hand_inventory[['SKU', 'AVAILABLE_ON_HAND_QTY', 'CURRENT_INVENTORY_POSITION']],
    on='SKU',
    how='left'
)
forecast_base['AVAILABLE_ON_HAND_QTY'] = forecast_base['AVAILABLE_ON_HAND_QTY'].fillna(0)
forecast_base['CURRENT_INVENTORY_POSITION'] = forecast_base['CURRENT_INVENTORY_POSITION'].fillna(0)

print(f"\nForecast base after merging on-hand inventory: {forecast_base.shape}")
print(f"\nSample forecast base:")
print(forecast_base.head(10))

Unique SKUs: 3884
Unique months: 14
Month range: 2025-11-01 00:00:00 to 2026-12-01 00:00:00

Forecast base after merging on-hand inventory: (54376, 5)

Sample forecast base:
          SKU      MONTH  UNIT DEMAND  AVAILABLE_ON_HAND_QTY  \
0   P20001-05 2025-11-01          458                      0   
1   P20001-24 2025-11-01          380                    147   
2   P20001-25 2025-11-01          345                      0   
3   P20001-23 2025-11-01          281                      0   
4  D104007-01 2025-11-01          126                      0   
5   P20001-36 2025-11-01          360                      0   
6  D104009-01 2025-11-01          136                      0   
7  D104008-01 2025-11-01           99                      0   
8  D104010-01 2025-11-01          105                      0   
9  D104011-01 2025-11-01          115                      0   

   CURRENT_INVENTORY_POSITION  
0                        -570  
1                         147  
2                        

In [159]:
# Merge with on-order monthly data
forecast_base = forecast_base.merge(
    on_order_monthly,
    on=['SKU', 'MONTH'],
    how='left'
)
forecast_base['ON_ORDER_QTY'] = forecast_base['ON_ORDER_QTY'].fillna(0)

# Sort by SKU and MONTH for proper processing
forecast_base = forecast_base.sort_values(['SKU', 'MONTH']).reset_index(drop=True)

print(f"\nForecast base after merging on-order data: {forecast_base.shape}")
print(f"\nSample forecast base:")
print(forecast_base.head(15))


Forecast base after merging on-order data: (54376, 6)

Sample forecast base:
                          SKU      MONTH  UNIT DEMAND  AVAILABLE_ON_HAND_QTY  \
0   000004556-004-FL-S-002-B1 2025-11-01            0                      0   
1   000004556-004-FL-S-002-B1 2025-12-01            0                      0   
2   000004556-004-FL-S-002-B1 2026-01-01            0                      0   
3   000004556-004-FL-S-002-B1 2026-02-01            0                      0   
4   000004556-004-FL-S-002-B1 2026-03-01            0                      0   
5   000004556-004-FL-S-002-B1 2026-04-01            0                      0   
6   000004556-004-FL-S-002-B1 2026-05-01            0                      0   
7   000004556-004-FL-S-002-B1 2026-06-01            0                      0   
8   000004556-004-FL-S-002-B1 2026-07-01            0                      0   
9   000004556-004-FL-S-002-B1 2026-08-01            0                      0   
10  000004556-004-FL-S-002-B1 2026-09-01  

## Calculate Expected Unit Sales

Expected Unit Sales = min(Unit Demand, Available Inventory)

Available Inventory for a given month = Available On-Hand + Cumulative On-Order that has arrived by that month

**Backorder/Pre-order Handling:**
- If CURRENT_INVENTORY_POSITION is negative, it indicates pre-orders/backorders exist
- Backorders are tracked month-to-month and persist until fully fulfilled
- When on-order inventory arrives, backorders are fulfilled FIRST before new sales
- Only inventory remaining after backorder fulfillment is available for new sales
- This ensures pre-sold commitments are honored before allocating inventory to new demand

In [ ]:
# For each SKU, track inventory month by month
forecast_base['AVAILABLE_INVENTORY'] = 0.0
forecast_base['EXPECTED_UNIT_SALES'] = 0.0
forecast_base['MISSED_DEMAND'] = 0.0
forecast_base['BACKORDERS_PENDING'] = 0.0  # Track backorders at start of each month
forecast_base['BACKORDERS_FULFILLED'] = 0.0  # Track backorders fulfilled each month

for sku in unique_skus:
    sku_mask = forecast_base['SKU'] == sku
    sku_data = forecast_base[sku_mask].copy().sort_values('MONTH').reset_index(drop=True)
    
    # Get starting inventory position
    starting_available = sku_data.iloc[0]['AVAILABLE_ON_HAND_QTY']
    starting_position = sku_data.iloc[0]['CURRENT_INVENTORY_POSITION']
    
    # Ensure starting available is non-negative
    starting_available = max(0, starting_available)
    
    # Track backorders/pre-orders (negative position means backorders exist)
    # If CURRENT_INVENTORY_POSITION is negative, that's the number of backorders to fulfill
    # Backorders persist across months until fully fulfilled
    backorders = abs(min(0, starting_position))
    
    # Starting available inventory (what we can sell now)
    # AVAILABLE_ON_HAND_QTY already excludes committed/backordered units
    # Ensure it's non-negative
    current_inventory = max(0, starting_available)
    
    # Process each month
    for i in range(len(sku_data)):
        row = sku_data.iloc[i]
        month = row['MONTH']
        
        # Add on-order inventory arriving this month
        on_order_this_month = max(0, row['ON_ORDER_QTY'])  # Ensure non-negative
        
                # Store backorders at start of month (before fulfillment)
        backorders_at_start = backorders
        backorders_fulfilled_this_month = 0
        
        # CRITICAL: If we have backorders, fulfill them FIRST with incoming on-order inventory
        # Backorders are pre-sold commitments, so they must be fulfilled before new sales
        # Backorders carry forward month-to-month until fully fulfilled
        if backorders > 0 and on_order_this_month > 0:
            # Fulfill as many backorders as possible with this month's on-order
            backorders_fulfilled_this_month = min(backorders, on_order_this_month)
            backorders -= backorders_fulfilled_this_month  # Reduce backorders (carries forward if not fully fulfilled)
            # Only the remaining on-order (after fulfilling backorders) is available for new sales
            available_from_on_order = on_order_this_month - backorders_fulfilled_this_month
            current_inventory += available_from_on_order
        elif backorders > 0:
            # We have backorders but no on-order this month - backorders carry forward
            # No inventory available for new sales this month
            available_from_on_order = 0
            # current_inventory remains unchanged (backorders still pending)
        else:
            # No backorders, so all on-order is available for new sales
            current_inventory += on_order_this_month
        
        # Ensure inventory never goes negative
        current_inventory = max(0, current_inventory)
        
        # Available inventory at start of month (after on-order arrives, backorders fulfilled)
        # This is what's available for NEW sales (not committed/backordered)
        # Ensure it's non-negative
        available_at_start = max(0, current_inventory)
        
                # Calculate forecast sales: min(demand, available inventory)
        # Both should be non-negative, so forecast_sales will be non-negative
        demand = max(0, row['UNIT DEMAND'])  # Ensure demand is non-negative
        
        # Check if SKU is MTO or DROPSHIP - these can fulfill full demand without inventory
        # Access from row (which is a pandas Series)
        is_mto = False
        is_dropship = False
        if 'IS_MTO' in row.index:
            is_mto_val = row['IS_MTO']
            is_mto = bool(is_mto_val) if pd.notna(is_mto_val) else False
        if 'IS_DROPSHIP' in row.index:
            is_dropship_val = row['IS_DROPSHIP']
            is_dropship = bool(is_dropship_val) if pd.notna(is_dropship_val) else False
        is_mto_or_dropship = is_mto or is_dropship
        
        if is_mto_or_dropship:
            # MTO/DROPSHIP products can fulfill full demand regardless of inventory
            forecast_sales = demand
        else:
            # Regular products are limited by available inventory
            forecast_sales = min(demand, available_at_start)
        
        # Ensure forecast sales is non-negative (should already be, but double-check)
        forecast_sales = max(0, forecast_sales)
        
        # Update inventory after sales
        current_inventory = max(0, available_at_start - forecast_sales)
        
                # Calculate missed demand (demand not captured by available inventory)
        missed_demand = max(0, demand - forecast_sales)
        
        # Store results
        idx = forecast_base[sku_mask].index[i]
        forecast_base.loc[idx, 'AVAILABLE_INVENTORY'] = available_at_start
        forecast_base.loc[idx, 'EXPECTED_UNIT_SALES'] = forecast_sales
        forecast_base.loc[idx, 'MISSED_DEMAND'] = missed_demand
        forecast_base.loc[idx, 'BACKORDERS_PENDING'] = backorders_at_start  # Backorders at start of month
        forecast_base.loc[idx, 'BACKORDERS_FULFILLED'] = backorders_fulfilled_this_month  # Backorders fulfilled this month

print("Calculated forecast unit sales and missed demand with proper inventory tracking")
print(f"\nSample results:")
print(forecast_base[['SKU', 'MONTH', 'UNIT DEMAND', 'EXPECTED_UNIT_SALES', 'MISSED_DEMAND', 'AVAILABLE_INVENTORY', 'BACKORDERS_PENDING', 'BACKORDERS_FULFILLED']].head(20))

# Show examples of SKUs with backorders being fulfilled over time
skus_with_backorders = forecast_base[forecast_base['BACKORDERS_PENDING'] > 0]['SKU'].unique()
if len(skus_with_backorders) > 0:
    print(f"\n=== Examples of backorder fulfillment (showing first 3 SKUs with backorders) ===")
    print(f"Total SKUs with backorders: {len(skus_with_backorders)}")
    for sku in skus_with_backorders[:3]:
        sku_data = forecast_base[forecast_base['SKU'] == sku].sort_values('MONTH')
        print(f"\nSKU: {sku}")
        # Show columns that exist
        display_cols = ['MONTH', 'BACKORDERS_PENDING', 'BACKORDERS_FULFILLED', 'ON_ORDER_QTY', 'AVAILABLE_INVENTORY', 'EXPECTED_UNIT_SALES']
        available_cols = [col for col in display_cols if col in sku_data.columns]
        print(sku_data[available_cols].to_string())
else:
    print("\nNo SKUs with backorders found in the data.")

## Set Today's Date

Set the reference date for calculating order feasibility based on lead times.

In [ ]:
# Set today's date as reference point for order feasibility 
today = datetime.now().date()
print(f"Today's date: {today}")
print(f"Using this date to determine if orders can be placed and received before forecast months")

## Calculate Potential Receipts and Forecast Unit Sales

Calculate POTENTIAL RECEIPTS based on lead times - inventory we could order today and receive before the 7th day of each forecast month.

Then calculate FORECAST UNIT SALES = min(UNIT DEMAND, AVAILABLE INVENTORY + POTENTIAL RECEIPTS)

In [ ]:
# Merge catalog lead times, planning status, and MTO/DROPSHIP flags with forecast base
forecast_base = forecast_base.merge(
    catalog_leadtimes[['SKU', 'PRODUCTION_TIME', 'PLANNING_STATUS', 'IS_MTO', 'IS_DROPSHIP']],
    on='SKU',
    how='left'
)
forecast_base['PRODUCTION_TIME'] = forecast_base['PRODUCTION_TIME'].fillna(0)
# Clean PLANNING_STATUS for consistent matching
forecast_base['PLANNING_STATUS'] = forecast_base['PLANNING_STATUS'].fillna('').astype(str).str.upper().str.strip()
# Fill missing MTO/DROPSHIP values with False
forecast_base['IS_MTO'] = forecast_base['IS_MTO'].fillna(False)
forecast_base['IS_DROPSHIP'] = forecast_base['IS_DROPSHIP'].fillna(False)

# Initialize new columns
forecast_base['POTENTIAL_RECEIPTS'] = 0.0
forecast_base['FORECAST_UNIT_SALES'] = 0.0

print(f"Merged catalog lead times and planning status. Forecast base shape: {forecast_base.shape}")
print(f"SKUs with production time data: {(forecast_base['PRODUCTION_TIME'] > 0).sum()} rows")
print(f"Rows with ARCHIVED status: {(forecast_base['PLANNING_STATUS'] == 'ARCHIVED').sum()}")
print(f"Rows with DISCONTINUED status: {(forecast_base['PLANNING_STATUS'] == 'DISCONTINUED').sum()}")
print(f"Rows with IS_MTO = TRUE: {forecast_base['IS_MTO'].sum()}")
print(f"Rows with IS_DROPSHIP = TRUE: {forecast_base['IS_DROPSHIP'].sum()}")
print(f"Rows with IS_MTO OR IS_DROPSHIP = TRUE: {(forecast_base['IS_MTO'] | forecast_base['IS_DROPSHIP']).sum()}")

In [ ]:
# Calculate potential receipts for each SKU/month combination
for idx, row in forecast_base.iterrows():
    sku = row['SKU']
    month = row['MONTH']
    # Ensure month is a datetime object
    if not isinstance(month, pd.Timestamp):
        month = pd.to_datetime(month)
    production_time = row['PRODUCTION_TIME']
    planning_status = row['PLANNING_STATUS']
    
    # Get base values
    unit_demand = max(0, row['UNIT DEMAND'])
    backorders_pending = max(0, row['BACKORDERS_PENDING'])
    available_inventory = max(0, row['AVAILABLE_INVENTORY'])
    on_order_qty = max(0, row['ON_ORDER_QTY'])
    expected_unit_sales = max(0, row['EXPECTED_UNIT_SALES'])
    
    # Check if SKU is MTO or DROPSHIP - these don't need inventory to fulfill demand
    is_mto = False
    is_dropship = False
    if 'IS_MTO' in row.index:
        is_mto_val = row['IS_MTO']
        is_mto = bool(is_mto_val) if pd.notna(is_mto_val) else False
    if 'IS_DROPSHIP' in row.index:
        is_dropship_val = row['IS_DROPSHIP']
        is_dropship = bool(is_dropship_val) if pd.notna(is_dropship_val) else False
    is_mto_or_dropship = is_mto or is_dropship
    
    # Check if SKU is ARCHIVED or DISCONTINUED - we won't reorder these
    is_archived_or_discontinued = (planning_status == 'ARCHIVED') or (planning_status == 'DISCONTINUED')
    
    if is_mto_or_dropship:
        # For MTO/DROPSHIP SKUs, they can fulfill full demand without inventory
        # No potential receipts needed (we don't order inventory for these)
        potential_receipts = 0
        forecast_unit_sales = unit_demand  # Full demand fulfillment
    elif is_archived_or_discontinued:
        # For archived/discontinued SKUs, no potential receipts and forecast = expected
        potential_receipts = 0
        forecast_unit_sales = expected_unit_sales
    else:
        # For active SKUs, calculate potential receipts based on lead times
        # Calculate when order would ship (today + production time)
        order_ship_date = today + timedelta(days=int(production_time))
        
        # Calculate the 7th day of the forecast month
        month_7th_day = month.replace(day=7).date()
        
        # Check if we can order and receive before the 7th day of the month
        if order_ship_date < month_7th_day:
            # Calculate inventory needed
            # Inventory needed = demand + backorders - (available + on-order)
            inventory_needed = unit_demand + backorders_pending - (available_inventory + on_order_qty)
            
            # Potential receipts is the amount we could order to meet the gap
            potential_receipts = max(0, inventory_needed)
        else:
            # Cannot order in time for this month
            potential_receipts = 0
        
        # Calculate forecast unit sales with potential receipts
        available_with_potential = available_inventory + potential_receipts
        forecast_unit_sales = min(unit_demand, max(0, available_with_potential))
        forecast_unit_sales = max(0, forecast_unit_sales)  # Ensure non-negative
    
    # Store results
    forecast_base.loc[idx, 'POTENTIAL_RECEIPTS'] = potential_receipts
    forecast_base.loc[idx, 'FORECAST_UNIT_SALES'] = forecast_unit_sales

print("Calculated potential receipts and forecast unit sales")
print(f"\nSample results:")
print(forecast_base[['SKU', 'MONTH', 'UNIT DEMAND', 'EXPECTED_UNIT_SALES', 'FORECAST_UNIT_SALES', 'POTENTIAL_RECEIPTS', 'AVAILABLE_INVENTORY', 'PLANNING_STATUS']].head(20))

# Summary statistics
archived_discontinued_mask = (forecast_base['PLANNING_STATUS'] == 'ARCHIVED') | (forecast_base['PLANNING_STATUS'] == 'DISCONTINUED')
print(f"\n=== Summary Statistics ===")
print(f"Total potential receipts: {forecast_base['POTENTIAL_RECEIPTS'].sum():,.0f}")
print(f"Total forecast unit sales: {forecast_base['FORECAST_UNIT_SALES'].sum():,.0f}")
print(f"Rows with potential receipts > 0: {(forecast_base['POTENTIAL_RECEIPTS'] > 0).sum()}")
print(f"\n=== Archived/Discontinued SKUs Handling ===")
print(f"Rows with ARCHIVED/DISCONTINUED status: {archived_discontinued_mask.sum()}")
print(f"Potential receipts for archived/discontinued: {forecast_base.loc[archived_discontinued_mask, 'POTENTIAL_RECEIPTS'].sum():,.0f}")
print(f"(Should be 0 - we don't reorder these SKUs)")

# Summary for MTO/DROPSHIP SKUs
mto_dropship_mask_base = forecast_base['IS_MTO'] | forecast_base['IS_DROPSHIP']
if mto_dropship_mask_base.sum() > 0:
    print(f"\n=== MTO/DROPSHIP SKUs Handling ===")
    print(f"Rows with MTO/DROPSHIP: {mto_dropship_mask_base.sum()}")
    print(f"Potential receipts for MTO/DROPSHIP: {forecast_base.loc[mto_dropship_mask_base, 'POTENTIAL_RECEIPTS'].sum():,.0f} (should be 0)")
    print(f"Expected sales for MTO/DROPSHIP: {forecast_base.loc[mto_dropship_mask_base, 'EXPECTED_UNIT_SALES'].sum():,.0f}")
    print(f"Forecast sales for MTO/DROPSHIP: {forecast_base.loc[mto_dropship_mask_base, 'FORECAST_UNIT_SALES'].sum():,.0f}")
    print(f"Demand for MTO/DROPSHIP: {forecast_base.loc[mto_dropship_mask_base, 'UNIT DEMAND'].sum():,.0f}")
    print(f"(MTO/DROPSHIP products can fulfill full demand without inventory constraints)")

## Create Final Output

Create the final output with the requested columns plus inventory and backorder tracking fields:
- SKU, MONTH, UNIT DEMAND, EXPECTED UNIT SALES, FORECAST UNIT SALES, MISSED DEMAND
- AVAILABLE INVENTORY, ON ORDER QTY, POTENTIAL RECEIPTS
- BACKORDERS PENDING, BACKORDERS FULFILLED

In [ ]:
# Create final output dataframe
# Include inventory and backorder tracking fields for validation
# Note: PLANNING_STATUS, IS_MTO, IS_DROPSHIP are included temporarily for calculations
final_output = forecast_base[['SKU', 'MONTH', 'UNIT DEMAND', 'EXPECTED_UNIT_SALES', 'FORECAST_UNIT_SALES', 'MISSED_DEMAND', 'PLANNING_STATUS', 'IS_MTO', 'IS_DROPSHIP',
                              'AVAILABLE_INVENTORY', 'ON_ORDER_QTY', 'POTENTIAL_RECEIPTS', 'BACKORDERS_PENDING', 'BACKORDERS_FULFILLED']].copy()

# Ensure UNIT DEMAND is non-negative (clean any negative demand values)
final_output['UNIT DEMAND'] = final_output['UNIT DEMAND'].clip(lower=0)

# Round all numeric fields to whole numbers (since we're dealing with units)
# IMPORTANT: After rounding, ensure forecast sales never exceeds demand and is never negative
final_output['EXPECTED_UNIT_SALES'] = final_output['EXPECTED_UNIT_SALES'].round().astype(int)
final_output['FORECAST_UNIT_SALES'] = final_output['FORECAST_UNIT_SALES'].round().astype(int)
final_output['MISSED_DEMAND'] = final_output['MISSED_DEMAND'].round().astype(int)
final_output['AVAILABLE_INVENTORY'] = final_output['AVAILABLE_INVENTORY'].round().astype(int)
final_output['ON_ORDER_QTY'] = final_output['ON_ORDER_QTY'].round().astype(int)
final_output['POTENTIAL_RECEIPTS'] = final_output['POTENTIAL_RECEIPTS'].round().astype(int)
final_output['BACKORDERS_PENDING'] = final_output['BACKORDERS_PENDING'].round().astype(int)
final_output['BACKORDERS_FULFILLED'] = final_output['BACKORDERS_FULFILLED'].round().astype(int)

# Ensure forecast sales is non-negative (should already be, but enforce it)
final_output['EXPECTED_UNIT_SALES'] = final_output['EXPECTED_UNIT_SALES'].clip(lower=0)

# Cap forecast sales at demand (in case rounding caused it to exceed)
# This ensures forecast sales never exceeds demand
final_output['EXPECTED_UNIT_SALES'] = final_output[['EXPECTED_UNIT_SALES', 'UNIT DEMAND']].min(axis=1)

# Recalculate missed demand based on capped forecast sales to ensure consistency
final_output['MISSED_DEMAND'] = (final_output['UNIT DEMAND'] - final_output['EXPECTED_UNIT_SALES']).clip(lower=0)

# Set MISSED_DEMAND = 0 for ARCHIVED/DISCONTINUED SKUs (we're not trying to meet demand beyond available inventory)
# Ensure PLANNING_STATUS is properly formatted
final_output['PLANNING_STATUS'] = final_output['PLANNING_STATUS'].fillna('').astype(str).str.upper().str.strip()

# Set missed demand to 0 for archived/discontinued SKUs
archived_discontinued_mask = (final_output['PLANNING_STATUS'] == 'ARCHIVED') | (final_output['PLANNING_STATUS'] == 'DISCONTINUED')
final_output.loc[archived_discontinued_mask, 'MISSED_DEMAND'] = 0

# Set missed demand to 0 for MTO/DROPSHIP SKUs (they can fulfill full demand)
# Fill missing values with False for boolean columns
final_output['IS_MTO'] = final_output['IS_MTO'].fillna(False)
final_output['IS_DROPSHIP'] = final_output['IS_DROPSHIP'].fillna(False)
mto_dropship_mask = final_output['IS_MTO'] | final_output['IS_DROPSHIP']
final_output.loc[mto_dropship_mask, 'MISSED_DEMAND'] = 0

# Drop temporary columns from final output (not needed in export)
final_output = final_output.drop(columns=['PLANNING_STATUS', 'IS_MTO', 'IS_DROPSHIP'], errors='ignore')

# Now rename columns for better readability
final_output = final_output.rename(columns={
    'EXPECTED_UNIT_SALES': 'EXPECTED UNIT SALES',
    'FORECAST_UNIT_SALES': 'FORECAST UNIT SALES',
    'MISSED_DEMAND': 'MISSED DEMAND',
    'AVAILABLE_INVENTORY': 'AVAILABLE INVENTORY',
    'ON_ORDER_QTY': 'ON ORDER QTY',
    'POTENTIAL_RECEIPTS': 'POTENTIAL RECEIPTS',
    'BACKORDERS_PENDING': 'BACKORDERS PENDING',
    'BACKORDERS_FULFILLED': 'BACKORDERS FULFILLED'
})

# Sort by SKU and MONTH
final_output = final_output.sort_values(['SKU', 'MONTH']).reset_index(drop=True)

print("Final output shape:", final_output.shape)
print(f"\nFinal output columns: {final_output.columns.tolist()}")
print(f"\nFinal output sample (first 20 rows):")
print(final_output.head(20))
print(f"\nFinal output summary:")
print(final_output.describe())

## Validation and Analysis

Let's check some statistics to validate the results.

In [ ]:
# Validation checks
print("=== Validation Checks ===\n")

# Check 1: Expected unit sales should never exceed demand
exceeds_demand_expected = final_output[final_output['EXPECTED UNIT SALES'] > final_output['UNIT DEMAND']]
if len(exceeds_demand_expected) > 0:
    print(f"⚠️  WARNING: {len(exceeds_demand_expected)} rows where expected unit sales exceed demand")
    print(exceeds_demand_expected.head())
else:
    print("✓ Expected unit sales never exceed demand")

# Check 2: Forecast unit sales should never exceed demand
exceeds_demand_forecast = final_output[final_output['FORECAST UNIT SALES'] > final_output['UNIT DEMAND']]
if len(exceeds_demand_forecast) > 0:
    print(f"⚠️  WARNING: {len(exceeds_demand_forecast)} rows where forecast unit sales exceed demand")
    print(exceeds_demand_forecast.head())
else:
    print("✓ Forecast unit sales never exceed demand")

# Check 3: Expected unit sales should be non-negative
negative_sales_expected = final_output[final_output['EXPECTED UNIT SALES'] < 0]
if len(negative_sales_expected) > 0:
    print(f"⚠️  WARNING: {len(negative_sales_expected)} rows with negative expected unit sales")
else:
    print("✓ All expected unit sales are non-negative")

# Check 4: Forecast unit sales should be non-negative
negative_sales_forecast = final_output[final_output['FORECAST UNIT SALES'] < 0]
if len(negative_sales_forecast) > 0:
    print(f"⚠️  WARNING: {len(negative_sales_forecast)} rows with negative forecast unit sales")
else:
    print("✓ All forecast unit sales are non-negative")

# Check 5: Verify SKU-MONTH combinations are unique (no duplicates)
sku_month_combinations = final_output.groupby(['SKU', 'MONTH']).size()
duplicate_combinations = sku_month_combinations[sku_month_combinations > 1]
if len(duplicate_combinations) > 0:
    print(f"\n⚠️  WARNING: Found {len(duplicate_combinations)} duplicate SKU-MONTH combinations")
    print("This indicates data quality issues - each SKU should only appear once per month")
    print(duplicate_combinations.head(10))
else:
    print(f"\n✓ All SKU-MONTH combinations are unique (no duplicates)")
    print(f"  Total unique SKU-MONTH combinations: {len(final_output)}")
    print(f"  Total unique SKUs: {final_output['SKU'].nunique()}")
    print(f"  Total unique months: {final_output['MONTH'].nunique()}")
    print(f"  Average months per SKU: {len(final_output) / final_output['SKU'].nunique():.1f}")

# Check 3: Summary statistics
print(f"\n=== Summary Statistics ===")
print(f"Total SKUs: {final_output['SKU'].nunique()}")
print(f"Total months: {final_output['MONTH'].nunique()}")
print(f"Total rows: {len(final_output)}")
print(f"\nTotal Unit Demand: {final_output['UNIT DEMAND'].sum():,.0f}")
print(f"Total Expected Unit Sales: {final_output['EXPECTED UNIT SALES'].sum():,.0f}")
print(f"Total Forecast Unit Sales: {final_output['FORECAST UNIT SALES'].sum():,.0f}")
print(f"Total Potential Receipts: {final_output['POTENTIAL RECEIPTS'].sum():,.0f}")
print(f"Total Missed Demand: {final_output['MISSED DEMAND'].sum():,.0f}")
print(f"\nExpected Sales Coverage: {(final_output['EXPECTED UNIT SALES'].sum() / final_output['UNIT DEMAND'].sum() * 100):.1f}%")
print(f"Forecast Sales Coverage: {(final_output['FORECAST UNIT SALES'].sum() / final_output['UNIT DEMAND'].sum() * 100):.1f}%")

# Summary of archived/discontinued SKUs handling
# Note: We need to check this before PLANNING_STATUS is dropped from final_output
# So we'll check in forecast_base instead
archived_discontinued_in_base = (forecast_base['PLANNING_STATUS'] == 'ARCHIVED') | (forecast_base['PLANNING_STATUS'] == 'DISCONTINUED')
if archived_discontinued_in_base.sum() > 0:
    print(f"\n=== Archived/Discontinued SKUs Summary ===")
    print(f"Total rows with ARCHIVED/DISCONTINUED status: {archived_discontinued_in_base.sum()}")
    print(f"Unique ARCHIVED/DISCONTINUED SKUs: {forecast_base.loc[archived_discontinued_in_base, 'SKU'].nunique()}")
    print(f"Total potential receipts for these SKUs: {forecast_base.loc[archived_discontinued_in_base, 'POTENTIAL_RECEIPTS'].sum():,.0f} (should be 0)")
    print(f"Total missed demand for these SKUs: {forecast_base.loc[archived_discontinued_in_base, 'MISSED_DEMAND'].sum():,.0f} (should be 0)")
    print(f"Note: For ARCHIVED/DISCONTINUED SKUs, we don't plan to reorder, so:")
    print(f"  - POTENTIAL RECEIPTS = 0")
    print(f"  - MISSED DEMAND = 0")
    print(f"  - FORECAST UNIT SALES = EXPECTED UNIT SALES")

# Summary of MTO/DROPSHIP SKUs handling
mto_dropship_in_base = forecast_base['IS_MTO'] | forecast_base['IS_DROPSHIP']
if mto_dropship_in_base.sum() > 0:
    print(f"\n=== MTO/DROPSHIP SKUs Summary ===")
    print(f"Total rows with MTO/DROPSHIP: {mto_dropship_in_base.sum()}")
    print(f"Unique MTO/DROPSHIP SKUs: {forecast_base.loc[mto_dropship_in_base, 'SKU'].nunique()}")
    print(f"Total potential receipts for these SKUs: {forecast_base.loc[mto_dropship_in_base, 'POTENTIAL_RECEIPTS'].sum():,.0f} (should be 0)")
    print(f"Total missed demand for these SKUs: {forecast_base.loc[mto_dropship_in_base, 'MISSED_DEMAND'].sum():,.0f} (should be 0)")
    print(f"Total expected sales for these SKUs: {forecast_base.loc[mto_dropship_in_base, 'EXPECTED_UNIT_SALES'].sum():,.0f}")
    print(f"Total forecast sales for these SKUs: {forecast_base.loc[mto_dropship_in_base, 'FORECAST_UNIT_SALES'].sum():,.0f}")
    print(f"Total demand for these SKUs: {forecast_base.loc[mto_dropship_in_base, 'UNIT DEMAND'].sum():,.0f}")
    print(f"Note: For MTO/DROPSHIP SKUs, they can fulfill full demand without inventory, so:")
    print(f"  - POTENTIAL RECEIPTS = 0 (no inventory needed)")
    print(f"  - MISSED DEMAND = 0 (all demand can be met)")
    print(f"  - EXPECTED UNIT SALES = UNIT DEMAND")
    print(f"  - FORECAST UNIT SALES = UNIT DEMAND")

# Check 4: Show some examples where demand is not fully met (missed demand)
print(f"\n=== Examples with highest missed demand ===")
missed_demand_examples = final_output[final_output['MISSED DEMAND'] > 0].copy()
missed_demand_examples = missed_demand_examples.sort_values('MISSED DEMAND', ascending=False)
print(f"Rows with missed demand: {len(missed_demand_examples)}")
print(missed_demand_examples.head(20))

# Check 5: Verify missed demand calculation
print(f"\n=== Verification: MISSED DEMAND should equal UNIT DEMAND - EXPECTED UNIT SALES ===")
verification = final_output.copy()
verification['CALCULATED_MISSED'] = verification['UNIT DEMAND'] - verification['EXPECTED UNIT SALES']
verification['DIFF'] = verification['MISSED DEMAND'] - verification['CALCULATED_MISSED']
mismatches = verification[verification['DIFF'] != 0]
if len(mismatches) > 0:
    print(f"⚠️  WARNING: {len(mismatches)} rows where MISSED DEMAND calculation doesn't match")
    print(mismatches.head())
else:
    print("✓ MISSED DEMAND calculation is correct")

# Check 6: Forecast unit sales should be >= expected unit sales (since it includes potential receipts)
forecast_less_than_expected = final_output[final_output['FORECAST UNIT SALES'] < final_output['EXPECTED UNIT SALES']]
if len(forecast_less_than_expected) > 0:
    print(f"\n⚠️  WARNING: {len(forecast_less_than_expected)} rows where FORECAST UNIT SALES < EXPECTED UNIT SALES")
    print("This shouldn't happen - forecast should include potential receipts")
    print(forecast_less_than_expected.head())
else:
    print(f"\n✓ FORECAST UNIT SALES >= EXPECTED UNIT SALES (as expected with potential receipts)")

## Proposed Receipts Configuration

Configuration parameters for calculating proposed receipts - what inventory to buy now based on coverage period, MOQ requirements, and smart tiered safety stock.

In [ ]:
# Configuration for Proposed Receipts Calculation

# Coverage period: How many weeks ahead to buy inventory for
COVERAGE_PERIOD_WEEKS = 12  # Default: 12 weeks

# Safety stock tier configuration
# Top tier: Top 25% of SKUs by revenue (75th percentile and above)
TOP_TIER_PERCENTILE = 0.75  # Top 25% of SKUs
TOP_TIER_SAFETY_STOCK_PCT = 0.20  # 20% safety stock

# Mid tier: 25-50% of SKUs by revenue (50th to 75th percentile)
MID_TIER_PERCENTILE = 0.50  # 50th percentile (bottom of mid tier)
MID_TIER_SAFETY_STOCK_PCT = 0.10  # 10% safety stock

# Bottom tier: Below 50th percentile gets no safety stock

# MOQ rounding strategy
MOQ_ROUNDING_STRATEGY = 'round_up'  # Options: 'round_up', 'round_to_multiple', 'skip_if_below'

print("=== Proposed Receipts Configuration ===")
print(f"Coverage Period: {COVERAGE_PERIOD_WEEKS} weeks")
print(f"Top Tier (Top 25%): {TOP_TIER_SAFETY_STOCK_PCT*100:.0f}% safety stock")
print(f"Mid Tier (25-50%): {MID_TIER_SAFETY_STOCK_PCT*100:.0f}% safety stock")
print(f"Bottom Tier (Below 50%): No safety stock")
print(f"MOQ Strategy: {MOQ_ROUNDING_STRATEGY}")

## Load MOQ and Retail Price Data

Extract minimum order quantities (MOQ) and retail prices from catalog data for proposed receipts calculation.

In [ ]:
# Extract MOQ and retail price from catalog
catalog_moq_price = catalog[['SKU', 'ITEM_MOQS', 'FULL_PRICE_RETAIL']].copy()

# Clean MOQ: convert to numeric, handle missing values (default to 1)
catalog_moq_price['ITEM_MOQS'] = pd.to_numeric(catalog_moq_price['ITEM_MOQS'], errors='coerce').fillna(1)
# Ensure MOQ is at least 1
catalog_moq_price['ITEM_MOQS'] = catalog_moq_price['ITEM_MOQS'].clip(lower=1)

# Clean retail price: convert to numeric, handle missing values (default to 0)
catalog_moq_price['FULL_PRICE_RETAIL'] = pd.to_numeric(catalog_moq_price['FULL_PRICE_RETAIL'], errors='coerce').fillna(0)
# Ensure retail price is non-negative
catalog_moq_price['FULL_PRICE_RETAIL'] = catalog_moq_price['FULL_PRICE_RETAIL'].clip(lower=0)

print(f"Loaded MOQ and retail price data for {len(catalog_moq_price)} SKUs")
print(f"SKUs with MOQ > 1: {(catalog_moq_price['ITEM_MOQS'] > 1).sum()}")
print(f"SKUs with retail price > 0: {(catalog_moq_price['FULL_PRICE_RETAIL'] > 0).sum()}")
print(f"\\nSample MOQ and price data:")
print(catalog_moq_price.head(10))

## Calculate Revenue and Rank SKUs

Calculate total forecasted revenue per SKU (demand × retail price) to identify top performers for smart safety stock allocation.

In [ ]:
# Merge retail price with forecast base to calculate revenue
forecast_with_price = forecast_base.merge(
    catalog_moq_price[['SKU', 'FULL_PRICE_RETAIL']],
    on='SKU',
    how='left'
)
forecast_with_price['FULL_PRICE_RETAIL'] = forecast_with_price['FULL_PRICE_RETAIL'].fillna(0)

# Calculate revenue per SKU-month (demand × retail price)
forecast_with_price['REVENUE'] = forecast_with_price['UNIT DEMAND'] * forecast_with_price['FULL_PRICE_RETAIL']

# Aggregate total revenue per SKU across all forecast months
sku_revenue = forecast_with_price.groupby('SKU')['REVENUE'].sum().reset_index()
sku_revenue = sku_revenue.rename(columns={'REVENUE': 'TOTAL_REVENUE'})

# Get PLANNING_STATUS for each SKU from forecast_base
# Get unique SKU-PLANNING_STATUS mapping (take first occurrence if multiple)
sku_planning_status = forecast_base[['SKU', 'PLANNING_STATUS']].drop_duplicates(subset='SKU')
sku_revenue = sku_revenue.merge(sku_planning_status, on='SKU', how='left')

# Clean PLANNING_STATUS to ensure consistent format
sku_revenue['PLANNING_STATUS'] = sku_revenue['PLANNING_STATUS'].fillna('').astype(str).str.upper().str.strip()

# Filter to only active SKUs (exclude ARCHIVED and DISCONTINUED) for threshold calculation
active_skus = sku_revenue[
    (sku_revenue['PLANNING_STATUS'] != 'ARCHIVED') & 
    (sku_revenue['PLANNING_STATUS'] != 'DISCONTINUED')
].copy()

# Sort active SKUs by revenue descending
active_skus = active_skus.sort_values('TOTAL_REVENUE', ascending=False).reset_index(drop=True)

# Calculate percentile thresholds using only active SKUs
# TOP_TIER_PERCENTILE = 0.75 means we want top 25% (above 75th percentile)
# MID_TIER_PERCENTILE = 0.50 means we want 25-50% (between 50th and 75th percentile)
total_active_skus = len(active_skus)
if total_active_skus > 0:
    # For top 25%: we want the top 25% of SKUs (indices 0 to top_25_count-1 when sorted descending)
    # Example: 100 SKUs, top 25% = 25 SKUs = indices 0-24
    # Threshold is the minimum revenue in top 25% (revenue at index 24)
    top_25_count = max(1, int(total_active_skus * (1 - TOP_TIER_PERCENTILE)))  # Top 25% = 1 - 0.75 = 0.25
    top_tier_threshold_idx = max(0, top_25_count - 1)  # Last index in top 25% (0-indexed)
    
    # For mid tier (25-50%): we want SKUs ranked 25% to 50%
    # Example: 100 SKUs, 25-50% = 25 SKUs = indices 25-49
    # Threshold is the minimum revenue in 25-50% range (revenue at index 49)
    mid_tier_count = max(1, int(total_active_skus * MID_TIER_PERCENTILE))  # 50% of SKUs
    mid_tier_threshold_idx = max(0, mid_tier_count - 1)  # Last index in bottom 50% (0-indexed)
    
    # Get revenue thresholds from active SKUs (sorted descending by revenue)
    # Top tier threshold: minimum revenue value in top 25%
    top_tier_revenue_threshold = active_skus.iloc[top_tier_threshold_idx]['TOTAL_REVENUE']
    
    # Mid tier threshold: minimum revenue value in bottom 50% (which is max in 25-50% range)
    mid_tier_revenue_threshold = active_skus.iloc[mid_tier_threshold_idx]['TOTAL_REVENUE']
else:
    # No active SKUs, set thresholds to 0
    top_tier_revenue_threshold = 0
    mid_tier_revenue_threshold = 0

# Sort all SKUs by revenue descending (for display and tier assignment)
sku_revenue = sku_revenue.sort_values('TOTAL_REVENUE', ascending=False).reset_index(drop=True)

# Assign tiers based on revenue thresholds (calculated from active SKUs only)
sku_revenue['SAFETY_STOCK_TIER'] = 'None'
sku_revenue['SAFETY_STOCK_PCT'] = 0.0

# Explicitly set ARCHIVED and DISCONTINUED SKUs to 'None' tier
archived_discontinued_mask = (sku_revenue['PLANNING_STATUS'] == 'ARCHIVED') | (sku_revenue['PLANNING_STATUS'] == 'DISCONTINUED')
sku_revenue.loc[archived_discontinued_mask, 'SAFETY_STOCK_TIER'] = 'None'
sku_revenue.loc[archived_discontinued_mask, 'SAFETY_STOCK_PCT'] = 0.0

# Assign tiers to active SKUs only
active_mask = ~archived_discontinued_mask
if active_mask.sum() > 0:
    sku_revenue.loc[active_mask & (sku_revenue['TOTAL_REVENUE'] >= top_tier_revenue_threshold), 'SAFETY_STOCK_TIER'] = 'Top 25%'
    sku_revenue.loc[active_mask & (sku_revenue['TOTAL_REVENUE'] >= top_tier_revenue_threshold), 'SAFETY_STOCK_PCT'] = TOP_TIER_SAFETY_STOCK_PCT
    
    sku_revenue.loc[active_mask & (sku_revenue['TOTAL_REVENUE'] >= mid_tier_revenue_threshold) & 
                    (sku_revenue['TOTAL_REVENUE'] < top_tier_revenue_threshold), 'SAFETY_STOCK_TIER'] = '25-50%'
    sku_revenue.loc[active_mask & (sku_revenue['TOTAL_REVENUE'] >= mid_tier_revenue_threshold) & 
                    (sku_revenue['TOTAL_REVENUE'] < top_tier_revenue_threshold), 'SAFETY_STOCK_PCT'] = MID_TIER_SAFETY_STOCK_PCT

print(f"=== SKU Revenue Ranking ===")
print(f"Total SKUs: {len(sku_revenue)}")
print(f"Active SKUs (used for threshold calculation): {total_active_skus}")
print(f"Archived/Discontinued SKUs (excluded from tiers): {archived_discontinued_mask.sum()}")
print(f"\\nTier Distribution:")
print(sku_revenue['SAFETY_STOCK_TIER'].value_counts())
print(f"\\nTop Tier Revenue Threshold (from active SKUs): ${top_tier_revenue_threshold:,.2f}")
print(f"Mid Tier Revenue Threshold (from active SKUs): ${mid_tier_revenue_threshold:,.2f}")
print(f"\\nTop 10 SKUs by Revenue:")
print(sku_revenue[['SKU', 'TOTAL_REVENUE', 'SAFETY_STOCK_TIER', 'SAFETY_STOCK_PCT', 'PLANNING_STATUS']].head(10))

## Calculate Proposed Receipts

Calculate what inventory to buy NOW based on:
- Coverage period (how many weeks ahead to plan for)
- Forecasted demand within coverage period
- Current available inventory and on-order inventory
- Smart tiered safety stock (based on revenue performance)
- MOQ (minimum order quantity) requirements

In [ ]:
# Calculate coverage period end date
coverage_period_end = today + timedelta(weeks=COVERAGE_PERIOD_WEEKS)
coverage_period_end_date = coverage_period_end

print(f"Coverage Period: {today} to {coverage_period_end_date} ({COVERAGE_PERIOD_WEEKS} weeks)")

# Merge MOQ and safety stock tier data with forecast base
forecast_base = forecast_base.merge(
    catalog_moq_price[['SKU', 'ITEM_MOQS']],
    on='SKU',
    how='left'
)
forecast_base['ITEM_MOQS'] = forecast_base['ITEM_MOQS'].fillna(1).clip(lower=1)

forecast_base = forecast_base.merge(
    sku_revenue[['SKU', 'SAFETY_STOCK_TIER', 'SAFETY_STOCK_PCT', 'TOTAL_REVENUE']],
    on='SKU',
    how='left'
)
forecast_base['SAFETY_STOCK_TIER'] = forecast_base['SAFETY_STOCK_TIER'].fillna('None')
forecast_base['SAFETY_STOCK_PCT'] = forecast_base['SAFETY_STOCK_PCT'].fillna(0.0)
forecast_base['TOTAL_REVENUE'] = forecast_base['TOTAL_REVENUE'].fillna(0.0)

# Initialize proposed receipts columns
proposed_receipts_data = []

# Process each SKU
unique_skus_for_proposed = forecast_base['SKU'].unique()

for sku in unique_skus_for_proposed:
    sku_data = forecast_base[forecast_base['SKU'] == sku].copy()
    sku_data = sku_data.sort_values('MONTH').reset_index(drop=True)
    
    # Get SKU attributes
    planning_status = sku_data.iloc[0]['PLANNING_STATUS']
    is_mto = bool(sku_data.iloc[0]['IS_MTO']) if pd.notna(sku_data.iloc[0]['IS_MTO']) else False
    is_dropship = bool(sku_data.iloc[0]['IS_DROPSHIP']) if pd.notna(sku_data.iloc[0]['IS_DROPSHIP']) else False
    is_mto_or_dropship = is_mto or is_dropship
    is_archived_or_discontinued = (planning_status == 'ARCHIVED') or (planning_status == 'DISCONTINUED')
    
    # Skip MTO/DROPSHIP and archived/discontinued SKUs
    if is_mto_or_dropship or is_archived_or_discontinued:
        continue
    
    # Get MOQ and safety stock tier
    moq = int(sku_data.iloc[0]['ITEM_MOQS'])
    safety_stock_tier = sku_data.iloc[0]['SAFETY_STOCK_TIER']
    safety_stock_pct = float(sku_data.iloc[0]['SAFETY_STOCK_PCT'])
    total_revenue = float(sku_data.iloc[0]['TOTAL_REVENUE'])
    
    # Get starting inventory position
    starting_available = max(0, float(sku_data.iloc[0]['AVAILABLE_ON_HAND_QTY']))
    starting_position = float(sku_data.iloc[0]['CURRENT_INVENTORY_POSITION'])
    backorders = abs(min(0, starting_position))
    
    # Calculate demand and track inventory consumption within coverage period
    coverage_demand = 0.0
    coverage_on_order = 0.0
    current_inventory = starting_available
    cumulative_shortfall = 0.0  # Track how much demand we can't cover
    
    # Track inventory month by month to properly account for consumption
    for idx, row in sku_data.iterrows():
        month = row['MONTH']
        if not isinstance(month, pd.Timestamp):
            month = pd.to_datetime(month)
        
        # Check if month is within coverage period
        month_date = month.date() if hasattr(month, 'date') else pd.to_datetime(month).date()
        if month_date > coverage_period_end_date:
            break
        
        # Get demand and on-order for this month
        unit_demand = max(0, float(row['UNIT DEMAND']))
        on_order_this_month = max(0, float(row['ON_ORDER_QTY']))
        
        # Add to totals
        coverage_demand += unit_demand
        coverage_on_order += on_order_this_month
        
        # Simulate inventory consumption: on-order arrives, then demand consumes inventory
        # On-order arrives at start of month
        current_inventory += on_order_this_month
        
        # Demand consumes available inventory
        demand_covered = min(unit_demand, current_inventory)
        current_inventory -= demand_covered
        shortfall_this_month = unit_demand - demand_covered
        cumulative_shortfall += shortfall_this_month
    
    # Calculate base need
    # Base need = cumulative shortfall (demand we couldn't cover) + any remaining backorders
    # If we have inventory left, it reduces the need
    base_need = cumulative_shortfall + backorders - max(0, current_inventory)
    base_need = max(0, base_need)  # Can't have negative need
    
    # Apply safety stock
    safety_stock_amount = base_need * safety_stock_pct if base_need > 0 else 0
    
    # Total need before MOQ adjustment
    total_need = base_need + safety_stock_amount
    
    # Round up to MOQ
    if total_need > 0:
        if MOQ_ROUNDING_STRATEGY == 'round_up':
            # Round up to nearest MOQ
            proposed_receipts = int(np.ceil(total_need / moq) * moq)
        elif MOQ_ROUNDING_STRATEGY == 'round_to_multiple':
            # Round up to nearest multiple of MOQ
            proposed_receipts = int(np.ceil(total_need / moq) * moq)
        else:  # skip_if_below
            # Only order if need is at least MOQ
            proposed_receipts = int(np.ceil(total_need / moq) * moq) if total_need >= moq else 0
    else:
        proposed_receipts = 0
    
    moq_adjustment = max(0, proposed_receipts - total_need)
    
    # Only include SKUs with proposed receipts > 0
    if proposed_receipts > 0:
        proposed_receipts_data.append({
            'SKU': sku,
            'PROPOSED_RECEIPTS': proposed_receipts,
            'BASE_NEED': int(round(base_need)),
            'SAFETY_STOCK_AMOUNT': int(round(safety_stock_amount)),
            'SAFETY_STOCK_TIER': safety_stock_tier,
            'SAFETY_STOCK_PCT': safety_stock_pct * 100,  # Convert to percentage
            'MOQ': moq,
            'MOQ_ADJUSTMENT': int(round(moq_adjustment)),
            'COVERAGE_PERIOD_END': coverage_period_end_date,
            'COVERAGE_DEMAND': int(round(coverage_demand)),
            'COVERAGE_ON_ORDER': int(round(coverage_on_order)),
            'STARTING_INVENTORY': int(round(current_inventory)),
            'BACKORDERS': int(round(backorders)),
            'TOTAL_REVENUE': total_revenue
        })

# Create proposed receipts dataframe
proposed_receipts_df = pd.DataFrame(proposed_receipts_data)

# Sort by proposed receipts descending
proposed_receipts_df = proposed_receipts_df.sort_values('PROPOSED_RECEIPTS', ascending=False).reset_index(drop=True)

print(f"\\n=== Proposed Receipts Summary ===")
print(f"Total SKUs with proposed receipts: {len(proposed_receipts_df)}")
print(f"Total proposed receipts: {proposed_receipts_df['PROPOSED_RECEIPTS'].sum():,} units")
print(f"\\nBy Safety Stock Tier:")
print(proposed_receipts_df.groupby('SAFETY_STOCK_TIER').agg({
    'SKU': 'count',
    'PROPOSED_RECEIPTS': 'sum'
}).rename(columns={'SKU': 'SKU_COUNT', 'PROPOSED_RECEIPTS': 'TOTAL_UNITS'}))
print(f"\\nTop 20 SKUs by Proposed Receipts:")
print(proposed_receipts_df.head(20))

## Export Proposed Receipts

Export proposed receipts to CSV file for purchasing decisions.

In [ ]:
# Export proposed receipts to CSV
proposed_receipts_output_file = '../data/proposed_receipts_output.csv'
proposed_receipts_df.to_csv(proposed_receipts_output_file, index=False)
print(f"✓ Proposed receipts exported to: {proposed_receipts_output_file}")
print(f"  Total SKUs: {len(proposed_receipts_df):,}")
print(f"  Total units to order: {proposed_receipts_df['PROPOSED_RECEIPTS'].sum():,}")

# Validation checks
print(f"\\n=== Validation ===")
print(f"✓ All proposed receipts are non-negative: {(proposed_receipts_df['PROPOSED_RECEIPTS'] >= 0).all()}")
print(f"✓ All proposed receipts meet or exceed MOQ: {(proposed_receipts_df['PROPOSED_RECEIPTS'] >= proposed_receipts_df['MOQ']).all()}")
print(f"✓ Safety stock tiers are correctly assigned:")
print(f"  - Top 25%: {len(proposed_receipts_df[proposed_receipts_df['SAFETY_STOCK_TIER'] == 'Top 25%'])} SKUs")
print(f"  - 25-50%: {len(proposed_receipts_df[proposed_receipts_df['SAFETY_STOCK_TIER'] == '25-50%'])} SKUs")
print(f"  - None: {len(proposed_receipts_df[proposed_receipts_df['SAFETY_STOCK_TIER'] == 'None'])} SKUs")

# Summary statistics by tier
print(f"\\n=== Summary by Tier ===")
tier_summary = proposed_receipts_df.groupby('SAFETY_STOCK_TIER').agg({
    'SKU': 'count',
    'PROPOSED_RECEIPTS': ['sum', 'mean'],
    'BASE_NEED': 'sum',
    'SAFETY_STOCK_AMOUNT': 'sum',
    'MOQ_ADJUSTMENT': 'sum'
}).round(0)
print(tier_summary)

## Diagnostic: BASE_NEED Calculation for Specific SKU

Trace through the BASE_NEED calculation step-by-step for a specific SKU to understand how it works.

In [ ]:
# Diagnostic: Trace BASE_NEED calculation for a specific SKU
diagnostic_sku = 'D148004-01'

print(f"=== BASE_NEED Calculation Diagnostic for SKU: {diagnostic_sku} ===\n")

# Check if SKU exists in forecast_base
if diagnostic_sku not in forecast_base['SKU'].values:
    print(f"SKU {diagnostic_sku} not found in forecast data.")
    print(f"Available SKUs (sample): {forecast_base['SKU'].unique()[:10].tolist()}")
else:
    # Get SKU data
    sku_data = forecast_base[forecast_base['SKU'] == diagnostic_sku].copy()
    sku_data = sku_data.sort_values('MONTH').reset_index(drop=True)
    
    # Get SKU attributes
    planning_status = sku_data.iloc[0]['PLANNING_STATUS']
    is_mto = bool(sku_data.iloc[0]['IS_MTO']) if pd.notna(sku_data.iloc[0]['IS_MTO']) else False
    is_dropship = bool(sku_data.iloc[0]['IS_DROPSHIP']) if pd.notna(sku_data.iloc[0]['IS_DROPSHIP']) else False
    is_archived_or_discontinued = (planning_status == 'ARCHIVED') or (planning_status == 'DISCONTINUED')
    
    print(f"SKU Attributes:")
    print(f"  Planning Status: {planning_status}")
    print(f"  Is MTO: {is_mto}")
    print(f"  Is Dropship: {is_dropship}")
    print(f"  Archived/Discontinued: {is_archived_or_discontinued}\n")
    
    if is_mto or is_dropship or is_archived_or_discontinued:
        print("This SKU is excluded from proposed receipts calculation.")
    else:
        # Get starting inventory
        starting_available = max(0, float(sku_data.iloc[0]['AVAILABLE_ON_HAND_QTY']))
        starting_position = float(sku_data.iloc[0]['CURRENT_INVENTORY_POSITION'])
        backorders = abs(min(0, starting_position))
        
        print(f"Starting Inventory Position:")
        print(f"  Available On-Hand: {starting_available:.0f} units")
        print(f"  Current Inventory Position: {starting_position:.0f} units")
        print(f"  Backorders: {backorders:.0f} units\n")
        
        # Get MOQ and safety stock tier
        if 'ITEM_MOQS' in sku_data.columns:
            moq = int(sku_data.iloc[0]['ITEM_MOQS'])
        else:
            moq = 1
        
        if 'SAFETY_STOCK_TIER' in sku_data.columns:
            safety_stock_tier = sku_data.iloc[0]['SAFETY_STOCK_TIER']
            safety_stock_pct = float(sku_data.iloc[0]['SAFETY_STOCK_PCT'])
        else:
            safety_stock_tier = 'None'
            safety_stock_pct = 0.0
        
        print(f"Ordering Parameters:")
        print(f"  MOQ: {moq} units")
        print(f"  Safety Stock Tier: {safety_stock_tier}")
        print(f"  Safety Stock %: {safety_stock_pct*100:.0f}%\n")
        
        # Calculate coverage period
        coverage_period_end = today + timedelta(weeks=COVERAGE_PERIOD_WEEKS)
        print(f"Coverage Period: {today} to {coverage_period_end} ({COVERAGE_PERIOD_WEEKS} weeks)\n")
        
        # Month-by-month simulation
        print("Month-by-Month Inventory Simulation:")
        print("-" * 80)
        print(f"{'Month':<12} {'Demand':<10} {'On-Order':<12} {'Inv Start':<12} {'Inv End':<12} {'Shortfall':<12}")
        print("-" * 80)
        
        current_inventory = starting_available
        cumulative_shortfall = 0.0
        coverage_demand = 0.0
        coverage_on_order = 0.0
        
        for idx, row in sku_data.iterrows():
            month = row['MONTH']
            if not isinstance(month, pd.Timestamp):
                month = pd.to_datetime(month)
            
            month_date = month.date() if hasattr(month, 'date') else pd.to_datetime(month).date()
            if month_date > coverage_period_end:
                break
            
            unit_demand = max(0, float(row['UNIT DEMAND']))
            on_order_this_month = max(0, float(row['ON_ORDER_QTY']))
            
            coverage_demand += unit_demand
            coverage_on_order += on_order_this_month
            
            # On-order arrives at start of month
            inventory_before_demand = current_inventory + on_order_this_month
            
            # Demand consumes inventory
            demand_covered = min(unit_demand, inventory_before_demand)
            current_inventory = inventory_before_demand - demand_covered
            shortfall_this_month = unit_demand - demand_covered
            cumulative_shortfall += shortfall_this_month
            
            month_str = month.strftime('%Y-%m') if hasattr(month, 'strftime') else str(month)[:7]
            print(f"{month_str:<12} {unit_demand:<10.0f} {on_order_this_month:<12.0f} {inventory_before_demand:<12.0f} {current_inventory:<12.0f} {shortfall_this_month:<12.0f}")
        
        print("-" * 80)
        print(f"{'TOTAL':<12} {coverage_demand:<10.0f} {coverage_on_order:<12.0f} {'':<12} {current_inventory:<12.0f} {cumulative_shortfall:<12.0f}")
        print()
        
        # Calculate base need
        base_need = cumulative_shortfall + backorders - max(0, current_inventory)
        base_need = max(0, base_need)
        
        print(f"BASE_NEED Calculation:")
        print(f"  Cumulative Shortfall (unmet demand): {cumulative_shortfall:.0f} units")
        print(f"  + Backorders: {backorders:.0f} units")
        print(f"  - Remaining Inventory: {max(0, current_inventory):.0f} units")
        print(f"  = BASE_NEED: {base_need:.0f} units\n")
        
        # Apply safety stock
        safety_stock_amount = base_need * safety_stock_pct if base_need > 0 else 0
        total_need = base_need + safety_stock_amount
        
        print(f"Safety Stock Calculation:")
        print(f"  BASE_NEED: {base_need:.0f} units")
        print(f"  × Safety Stock % ({safety_stock_tier}): {safety_stock_pct*100:.0f}%")
        print(f"  = Safety Stock Amount: {safety_stock_amount:.0f} units\n")
        
        print(f"Total Need (before MOQ):")
        print(f"  BASE_NEED: {base_need:.0f} units")
        print(f"  + Safety Stock: {safety_stock_amount:.0f} units")
        print(f"  = Total Need: {total_need:.0f} units\n")
        
        # MOQ rounding
        if total_need > 0:
            proposed_receipts = int(np.ceil(total_need / moq) * moq)
            moq_adjustment = max(0, proposed_receipts - total_need)
        else:
            proposed_receipts = 0
            moq_adjustment = 0
        
        print(f"MOQ Adjustment:")
        print(f"  Total Need: {total_need:.0f} units")
        print(f"  MOQ: {moq} units")
        print(f"  Rounded Up: {proposed_receipts:.0f} units")
        print(f"  MOQ Adjustment: {moq_adjustment:.0f} units\n")
        
        print(f"FINAL PROPOSED_RECEIPTS: {proposed_receipts:.0f} units")

## Export Results

Save the final output to a CSV file.

In [ ]:
# Export to CSV
output_file = '../data/sales_forecast_output.csv'
final_output.to_csv(output_file, index=False)
print(f"✓ Results exported to: {output_file}")
print(f"  Total rows: {len(final_output):,}")